In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.dates as mdates

In [ ]:
train = pd.read_csv("../input/covid19-global-forecasting-week-3/train.csv")
train.head()

In [ ]:
train_data=train.drop('Province_State',axis=1)
train_data.head()

In [ ]:
test = pd.read_csv("../input/covid19-global-forecasting-week-3/test.csv")
test.head()

In [ ]:
test_data = test.drop('Province_State',axis=1)
test_data.head()

In [ ]:
bycountry = train_data.groupby('Country_Region')['ConfirmedCases'].max().sort_values(ascending=False).to_frame().reset_index()
bycountry.head(10)

In [ ]:
Top_10_Countries = bycountry.head(10)
fig = px.bar(Top_10_Countries, x='Country_Region', y='ConfirmedCases',color='ConfirmedCases')
fig.show()

In [ ]:
italy = train_data[train_data['Country_Region']=='Italy']
italy.head()

In [ ]:
fig = go.Figure(data=go.Scatter(x=italy['Date'], y=italy['ConfirmedCases'],
                                mode='lines+markers',
                                hovertemplate = "Cases: %{y}<br> %{x}<extra></extra>",
                                showlegend = False
                               ))
fig.show()

In [ ]:
confirmedcases_by_country = train_data.groupby(['Date','Country_Region'])['ConfirmedCases'].sum()\
.reset_index().set_index('Date')

df_countries_confirmed = confirmedcases_by_country[confirmedcases_by_country['Country_Region'].isin(Top_10_Countries.Country_Region)]


plt.figure(figsize=(16,12))
ax = sns.lineplot(x=df_countries_confirmed.index, y="ConfirmedCases", hue="Country_Region", data=df_countries_confirmed,palette='muted').set_title('Cumulative line')
plt.legend(loc=2, prop={'size': 12})
plt.title('Cumulative trend plot for Confirmed Cases')
plt.xticks(rotation=90);

Features Selection

In [ ]:
test_data['Date']=pd.to_datetime(test_data['Date'])
train_data['Date']=pd.to_datetime(train_data['Date'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
train_data['Country_Region']=LE.fit_transform(train_data['Country_Region'])
test_data['Country_Region']=LE.transform(test_data['Country_Region'])

train_data.loc[:, 'Date'] = train_data.Date.dt.strftime("%m%d")
train_data["Date"]  = train_data["Date"].astype(int)

test_data.loc[:, 'Date'] = test_data.Date.dt.strftime("%m%d")
test_data["Date"]  = test_data["Date"].astype(int)

In [ ]:
#Select feature column names and target variable we are going to use for training
features=['Date','Country_Region']
target1 = 'ConfirmedCases'

#This is input which our classifier will use as an input.
train_data[features].head(10)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define the model
random_forest = RandomForestClassifier(n_estimators=100, max_samples=0.8,
                        random_state=2)
# Train model
random_forest.fit(train_data[features],train_data[target1])

In [ ]:
predict = random_forest.predict(test_data[features])
predict

In [ ]:
target2='Fatalities'

In [ ]:
# Define the model
random_forest2 = RandomForestClassifier(n_estimators=100, max_samples=0.8,
                        random_state=1)
# Train model
random_forest2.fit(train_data[features],train_data[target2])

In [ ]:
#Make predictions using the features from the test data set
predict2 = random_forest2.predict(test_data[features])

print(predict2[0:50])

In [ ]:
#Create a  output DataFrame
submission = pd.DataFrame({'ForecastId':test_data['ForecastId'],'ConfirmedCases':predict,'Fatalities':predict2})
                        

#Visualize the first 10 rows
submission.head(10)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Define the model
de = DecisionTreeClassifier(max_depth=100,
                        random_state=3)
# Train model
de.fit(train_data[features],train_data[target1])


In [ ]:
predictd = de.predict(test_data[features])
predictd

In [ ]:
target2='Fatalities'

In [ ]:
# Define the model
de2 = DecisionTreeClassifier(max_depth=100,
                        random_state=1)
# Train model
de2.fit(train_data[features],train_data[target2])

In [ ]:
#Make predictions using the features from the test data set
predictd2 = random_forest2.predict(test_data[features])

print(predictd2[0:50])

In [ ]:
submission = pd.DataFrame({'ForecastId':test_data['ForecastId'],'ConfirmedCases':predictd,'Fatalities':predictd2})
                        

#Visualize the first 10 rows
submission.head(10)

In [ ]:
output_file = 'submission.csv'

submission.to_csv(output_file,index=False)

print('Saved file: ' + filename)